In [ ]:
from lib.measurement import MultiQubitMeasurement
from lib.formatting import format_bytes, bitstring_to_filestring
from lib.state_io import save_state_vector

####

from pathlib import Path
from time import time

import jax.numpy as jnp
from jax import random


rng_seed = 42

data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

print(f"Random seed is {rng_seed}")
print(f"Data will be saved to {data_dir.resolve()}")

In [7]:
#### PHASE AUGMENTED W STATE

num_qubits = 8

state_dim = 1 << num_qubits                 # bit shifting a ...0001 bitstring is the same as 2**num_qubits

rng_phase = random.PRNGKey(rng_seed)
one_hot_thetas = random.uniform(rng_phase, shape=(num_qubits,), minval=0.0, maxval=2 * jnp.pi)
one_hot_amplitudes = jnp.exp(1j * one_hot_thetas) / jnp.sqrt(num_qubits)

one_hot_indices = 1 << jnp.arange(num_qubits - 1, -1, -1)   # we shift a single bit left with descending ints

w_aug = jnp.zeros(state_dim, dtype=jnp.complex64).at[one_hot_indices].set(one_hot_amplitudes)

print("First 10 amplitudes:")
for idx in range(10):
    print(f"{idx:0{num_qubits}b}: {w_aug[idx]:.8f}")

print(f"\nSize of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

First 10 amplitudes:
00000000: 0.00000000+0.00000000j
00000001: 0.35348165+0.00712282j
00000010: 0.34222963+0.08876304j
00000011: 0.00000000+0.00000000j
00000100: -0.07679566+0.34511220j
00000101: 0.00000000+0.00000000j
00000110: 0.00000000+0.00000000j
00000111: 0.00000000+0.00000000j
00001000: 0.02867402-0.35238871j
00001001: 0.00000000+0.00000000j

Size of state vector in memory: 2.00 KB 



In [8]:
w_amp_file_name = f"w_aug_{num_qubits}.txt"
w_amp_file_path = data_dir / w_amp_file_name
save_state_vector(w_aug, w_amp_file_path)

State vector written to data/w_aug_8.txt (8960 bytes)


In [9]:
measurement_bases = []

# all Z
measurement_bases.append(['Z'] * num_qubits)

# sliding XX window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'X'
    measurement_bases.append(basis_list)

# sliding XY window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'Y'
    measurement_bases.append(basis_list)


for i, basis in enumerate(measurement_bases):
    print(f"Basis {i:2d}: {''.join(basis)}")

Basis  0: ZZZZZZZZ
Basis  1: XXZZZZZZ
Basis  2: ZXXZZZZZ
Basis  3: ZZXXZZZZ
Basis  4: ZZZXXZZZ
Basis  5: ZZZZXXZZ
Basis  6: ZZZZZXXZ
Basis  7: ZZZZZZXX
Basis  8: XYZZZZZZ
Basis  9: ZXYZZZZZ
Basis 10: ZZXYZZZZ
Basis 11: ZZZXYZZZ
Basis 12: ZZZZXYZZ
Basis 13: ZZZZZXYZ
Basis 14: ZZZZZZXY


In [11]:
samples_per_basis = 6400
rng_samples = random.PRNGKey(rng_seed)

for _, measurement_dirs in enumerate(measurement_bases):
    measurement_dir_str = ''.join(measurement_dirs)
    filename = data_dir / f"w_aug_{measurement_dir_str}_{samples_per_basis}.txt"

    measurement = MultiQubitMeasurement(measurement_dirs)

    start = time()
    samples = measurement.sample_state(w_aug, samples_per_basis, rng=rng_samples)
    print(f"Sampling {samples_per_basis} samples took {time() - start:.2f} seconds.")

    with open(filename, 'w') as f_out:
        for bitstring in samples:
            formatted = bitstring_to_filestring(bitstring, measurement_dirs)
            f_out.write(formatted + "\n")

    print(f"Stored {samples_per_basis} samples to {filename.name}.\n")

Constructing basis ZZZZZZZZ: 100%|██████████| 256/256 [00:00<00:00, 2596.53it/s]


Sampling 6400 samples took 0.02 seconds.
Stored 6400 samples to w_aug_ZZZZZZZZ_6400.txt.



Constructing basis XXZZZZZZ: 100%|██████████| 256/256 [00:00<00:00, 2586.31it/s]

Sampling 6400 samples took 0.02 seconds.


Stored 6400 samples to w_aug_XXZZZZZZ_6400.txt.



...

Sampling 6400 samples took 0.02 seconds.


Stored 6400 samples to w_aug_ZZZZZXYZ_6400.txt.



Constructing basis ZZZZZZXY: 100%|██████████| 256/256 [00:00<00:00, 2654.94it/s]

Sampling 6400 samples took 0.02 seconds.


Stored 6400 samples to w_aug_ZZZZZZXY_6400.txt.

